# Блокнот для сбора данных в единный фрейм, отбрасывание неправильно загруженных данных и неподходящих по дискретизации.

In [1]:
import pandas as pd
import os
import re

folder_path = "../data/external/features"
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
start_date_list = list()
end_date_list = list()
dataframes = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    if len(df) == 0:
        continue
    start_index = file.find("indicator_") + len("indicator_")
    end_index = file.rfind(".csv")
    substring = file[start_index:end_index].rstrip("_")

    df["Indicator"] = substring
    df["Filename"] = file

    df['DateTime'] = pd.to_datetime(df['DateTime'])
    df['DateTime'] = df['DateTime'].dt.normalize()
    df['DateTime'] = df['DateTime'].dt.to_period('M').dt.to_timestamp()

    start_time = df['DateTime'].min()
    end_time = df['DateTime'].max()
    if start_time > pd.to_datetime("2000-01-01") or end_time < pd.to_datetime("2025-01-01"):
        continue 
    start_date_list.append(start_time)
    end_date_list.append(end_time)

    df = df[["DateTime", "Close", "Frequency", "Indicator", "Filename"]].copy()

    dataframes.append(df)
    del df

df = pd.read_csv('../data/external/inflation_target/historical_country_United_States_indicator_inflation_rate.csv')
df.rename(columns={"Value" : "Close"}, inplace=True)
df["Indicator"] = "inflation_rate"
df["Filename"] = "historical_country_United_States_indicator_inflation_rate.csv"
df['DateTime'] = pd.to_datetime(df['DateTime'])
df['DateTime'] = df['DateTime'].dt.normalize()
df['DateTime'] = df['DateTime'].dt.to_period('M').dt.to_timestamp()
df = df[["DateTime", "Close", "Frequency", "Indicator", "Filename"]].copy()
dataframes.append(df)
del df 

full_data = pd.concat(dataframes, ignore_index=True)
full_data.rename(columns={"DateTime" : "Date"}, inplace=True)
full_data = full_data[full_data.Date >= pd.to_datetime("2000-07-01")].copy()

In [2]:
print(len(start_date_list), len(end_date_list), end="\n\n")
print(set(start_date_list), end="\n\n")
print(set(end_date_list))

176 176

{Timestamp('1981-12-01 00:00:00'), Timestamp('1914-01-01 00:00:00'), Timestamp('1997-02-01 00:00:00'), Timestamp('1992-03-01 00:00:00'), Timestamp('1964-01-01 00:00:00'), Timestamp('1996-01-01 00:00:00'), Timestamp('1976-06-01 00:00:00'), Timestamp('1983-09-01 00:00:00'), Timestamp('1969-01-01 00:00:00'), Timestamp('1982-06-01 00:00:00'), Timestamp('1919-02-01 00:00:00'), Timestamp('1959-01-01 00:00:00'), Timestamp('1938-12-01 00:00:00'), Timestamp('1991-01-01 00:00:00'), Timestamp('1952-11-01 00:00:00'), Timestamp('1955-01-01 00:00:00'), Timestamp('1964-02-01 00:00:00'), Timestamp('1963-02-01 00:00:00'), Timestamp('1993-05-01 00:00:00'), Timestamp('1968-05-01 00:00:00'), Timestamp('1967-01-01 00:00:00'), Timestamp('1993-01-01 00:00:00'), Timestamp('1971-04-01 00:00:00'), Timestamp('1954-01-01 00:00:00'), Timestamp('1994-01-01 00:00:00'), Timestamp('1992-02-01 00:00:00'), Timestamp('1942-01-01 00:00:00'), Timestamp('1978-01-01 00:00:00'), Timestamp('1987-12-01 00:00:00'), Time

In [3]:
print(f"Количество файлов: {len(set(csv_files))}.")
print(f"Количество загруженных csv файлов: {len(dataframes) - 1}.")
print(f"Данные загружены из {len(full_data.Filename.unique())} файлов.")

if len(full_data.Filename.unique()) <= len(set(csv_files)):
    print(f"При загрузке отброшены файлы: {set(csv_files) - set(full_data.Filename.unique())}")

Количество файлов: 293.
Количество загруженных csv файлов: 176.
Данные загружены из 177 файлов.
При загрузке отброшены файлы: {'historical_country_united_states_indicator_home_ownership_rate_.csv', 'historical_country_united_states_indicator_case_shiller_home_price_index_mom_.csv', 'historical_country_united_states_indicator_full_year_gdp_growth_.csv', 'historical_country_united_states_indicator_gdp_from_services_.csv', 'historical_country_united_states_indicator_producer_price_inflation_mom_.csv', 'historical_country_united_states_indicator_gross_national_product_.csv', 'historical_country_united_states_indicator_gdp_deflator_.csv', 'historical_country_united_states_indicator_productivity_.csv', 'historical_country_united_states_indicator_case_shiller_home_price_index_yoy_.csv', 'historical_country_united_states_indicator_gdp_from_utilities_.csv', 'historical_country_united_states_indicator_debt_balance_total_.csv', 'historical_country_united_states_indicator_changes_in_inventories_.c

In [4]:
for name in csv_files:
    if "inflation" in name:
        print(name)

historical_country_united_states_indicator_inflation_expectations_.csv
historical_country_united_states_indicator_energy_inflation_.csv
historical_country_united_states_indicator_michigan_inflation_expectations_.csv
historical_country_united_states_indicator_services_inflation_.csv
historical_country_united_states_indicator_michigan_5_year_inflation_expectations_.csv
historical_country_united_states_indicator_rent_inflation_.csv
historical_country_united_states_indicator_food_inflation_.csv
historical_country_united_states_indicator_core_inflation_rate_.csv
historical_country_united_states_indicator_core_inflation_rate_mom_.csv
historical_country_united_states_indicator_producer_price_inflation_mom_.csv
historical_country_united_states_indicator_inflation_rate_mom_.csv


In [5]:
print(full_data.shape, end="\n\n")
print(full_data.dtypes, end="\n\n")
print(full_data.describe, end="\n\n")
print(full_data.isna().sum())

(111698, 5)

Date         datetime64[ns]
Close               float64
Frequency            object
Indicator            object
Filename             object
dtype: object

<bound method NDFrame.describe of              Date      Close Frequency         Indicator  \
54     2000-07-01  4947731.0   Monthly  tourist_arrivals   
55     2000-08-01  4650387.0   Monthly  tourist_arrivals   
56     2000-09-01  3874868.0   Monthly  tourist_arrivals   
57     2000-10-01  3654971.0   Monthly  tourist_arrivals   
58     2000-11-01  3206700.0   Monthly  tourist_arrivals   
...           ...        ...       ...               ...   
212752 2024-11-01        2.7   Monthly    inflation_rate   
212753 2024-12-01        2.9   Monthly    inflation_rate   
212754 2025-01-01        3.0   Monthly    inflation_rate   
212755 2025-02-01        2.8   Monthly    inflation_rate   
212756 2025-03-01        2.4   Monthly    inflation_rate   

                                                 Filename  
54      historica

In [6]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 111698 entries, 54 to 212756
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   Date       111698 non-null  datetime64[ns]
 1   Close      111698 non-null  float64       
 2   Frequency  111698 non-null  object        
 3   Indicator  111698 non-null  object        
 4   Filename   111698 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 5.1+ MB


In [7]:
full_data = full_data[full_data.Frequency != "Yearly"].copy()
full_data.rename(columns={"Close" : "Value"}, inplace=True)
del full_data["Filename"]

In [8]:
full_data.head()

,Date,Value,Frequency,Indicator
54,2000-07-01,4947731.0,Monthly,tourist_arrivals
55,2000-08-01,4650387.0,Monthly,tourist_arrivals
56,2000-09-01,3874868.0,Monthly,tourist_arrivals
57,2000-10-01,3654971.0,Monthly,tourist_arrivals
58,2000-11-01,3206700.0,Monthly,tourist_arrivals


In [9]:
wide_full_data = full_data.pivot_table(index='Date', columns='Indicator', values='Value', aggfunc='mean').reset_index()

In [10]:
start = wide_full_data['Date'].min()
end = wide_full_data['Date'].max()
expected = pd.date_range(start=start, end=end, freq='MS')  # MS = Month Start
actual = pd.Series(wide_full_data['Date'].unique()).sort_values()
missing = expected.difference(actual)

if missing.empty:
    print("Все месяцы присутствуют.")
else:
    print("Пропущены месяцы:")
    print(missing)

wide_full_data.columns.name = None
wide_full_data = wide_full_data.set_index('Date')
wide_full_data = wide_full_data.sort_index() 

Все месяцы присутствуют.


In [11]:
stats = wide_full_data.isna().sum()/len(wide_full_data)
stats[stats > 0.1]

Series([], dtype: float64)

In [12]:
wide_full_data.isna().sum().sort_values()

15_year_mortgage_rate                   0
continuing_jobless_claims               0
philadelphia_fed_manufacturing_index    0
nahb_housing_market_index               0
crude_oil_rigs                          0
                                       ..
coincident_index                        3
tourist_arrivals                        3
national_home_price_index               3
crude_oil_production                    3
auto_exports                            3
Length: 172, dtype: int64

In [13]:
wide_full_data = wide_full_data.interpolate(method='time', limit_area="inside")

for col in wide_full_data.columns:
    series = wide_full_data[col]
    num_tail_nans = series[::-1].isna().sum()
    if num_tail_nans > 0:
        wide_full_data[col] = series.shift(int(num_tail_nans))

wide_full_data = wide_full_data[wide_full_data.index >= pd.to_datetime("2001-01-01")]

In [14]:
wide_full_data.isna().sum().sort_values()

15_year_mortgage_rate                     0
michigan_5_year_inflation_expectations    0
michigan_consumer_expectations            0
michigan_current_economic_conditions      0
michigan_inflation_expectations           0
                                         ..
export_prices                             0
export_prices_mom                         0
export_prices_yoy                         0
effective_federal_funds_rate              0
youth_unemployment_rate                   0
Length: 172, dtype: int64

In [15]:
wide_full_data

,15_year_mortgage_rate,30_year_mortgage_rate,auto_exports,average_house_prices,average_mortgage_size,balance_of_trade,bank_lending_rate,banks_balance_sheet,building_permits,building_permits_mom,...,unemployed_persons,unemployment_rate,used_car_prices_mom,used_car_prices_yoy,wage_growth,wages,wages_in_manufacturing,weekly_crude_oil_production,wholesale_inventories,youth_unemployment_rate
Date,,,,,,,,,,,,,,,,,,,,,
2001-01-01,6.6400,7.0325,67.0,208100.0,163.857143,-32.242,9.50,6192.220,1543.0,-4.4,...,5634.0,3.9,0.2,1.8,7.02,14.29,14.50,5929.434783,0.0,9.2
2001-02-01,6.6400,7.0500,52.1,209000.0,162.756522,-32.258,9.05,6143.275,1699.0,10.1,...,6023.0,4.2,0.0,2.5,5.52,14.29,14.48,5884.750000,-0.1,9.6
2001-03-01,6.5080,6.9520,47.0,211000.0,160.110000,-35.202,8.50,6163.050,1656.0,-2.5,...,6089.0,4.2,0.1,2.2,4.92,14.36,14.55,5874.227273,0.1,9.6
2001-04-01,6.5950,7.0775,44.2,210200.0,159.509091,-29.300,8.32,6224.075,1659.0,0.2,...,6141.0,4.3,-0.7,0.9,4.79,14.42,14.59,5867.380952,0.0,9.8
2001-05-01,6.6960,7.1640,52.2,205500.0,159.542857,-32.811,7.80,6207.180,1666.0,0.4,...,6271.0,4.4,-0.2,0.9,4.68,14.44,14.63,5814.217391,0.5,10.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-01,5.9300,6.7150,72.9,478400.0,402.873000,-73.730,7.75,23721.200,1493.0,5.2,...,7121.0,4.2,1.3,0.2,5.00,30.58,28.22,13598.250000,-0.1,9.4
2025-01-01,6.1640,6.9580,55.4,509700.0,400.930000,-78.240,7.50,23693.400,1482.0,-0.7,...,6886.0,4.1,-0.3,0.4,5.12,30.67,28.33,13447.800000,-0.4,9.0
2025-02-01,6.0300,6.8425,68.7,510000.0,403.416000,-98.060,7.50,23766.975,1473.0,-0.6,...,6849.0,4.0,0.4,0.8,5.04,30.80,28.58,13500.250000,0.8,9.0


In [18]:
columns_with_double_underscore = [col for col in wide_full_data.columns if '__' in col]
columns_with_triple_underscore = [col for col in wide_full_data.columns if '___' in col]
print(columns_with_double_underscore)
print(columns_with_triple_underscore)

['continued_jobless_claims___federal_workers', 'jobless_claims___federal_workers']
['continued_jobless_claims___federal_workers', 'jobless_claims___federal_workers']


In [19]:
wide_full_data.columns = [col.replace('___', '_').replace('__', '_') for col in wide_full_data.columns]

In [20]:
columns_with_double_underscore = [col for col in wide_full_data.columns if '__' in col]
columns_with_triple_underscore = [col for col in wide_full_data.columns if '___' in col]
print(columns_with_double_underscore)
print(columns_with_triple_underscore)

[]
[]


In [21]:
wide_full_data.to_csv("../data/interim/full_collected_data.csv")